In [2]:
# !pip install gym-trading-env
import numpy as np
import pandas as pd
import datetime

import matplotlib.pyplot as plt
%matplotlib inline
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
from statsmodels.tsa.stattools import acf, pacf

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import ElasticNet



from adtk.visualization import plot
import ml_metrics as metrix

import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
from gym_trading_env.downloader import download
import seaborn as sns

import warnings
warnings.simplefilter('ignore')

In [4]:
# df.to_pickle('df_.pkl')
df = pd.read_pickle('df_.pkl')


# Nostradamus  MODEL 1Hour Prediction

Определяем количество элементов вручную которые влияют на наш прогноз

In [59]:
p = 35057 # место с которого мы ничего не знаем
df.head(p)

,open,high,low,close,volume,date_close,date_open,VWAP,seasonal_index,trend_index,resid_index,UnixH
date_close,,,,,,,,,,,,
2020-02-10 00:00:00,10126.22,10160.00,10119.78,10151.75,1786.197512,2020-02-10 00:00:00,2020-02-09 23:00:00,10151.750000,1.171449,3946.090128,2.196092,439248.0
2020-02-10 01:00:00,10151.72,10188.00,10150.00,10184.19,1767.194315,2020-02-10 01:00:00,2020-02-10 00:00:00,10167.883257,1.172082,3946.677363,2.198067,439249.0
2020-02-10 02:00:00,10184.19,10184.40,10130.00,10133.32,1151.821611,2020-02-10 02:00:00,2020-02-10 01:00:00,10159.422279,1.171743,3947.264599,2.196547,439250.0
2020-02-10 03:00:00,10133.33,10147.26,10080.00,10120.59,2238.188839,2020-02-10 03:00:00,2020-02-10 02:00:00,10146.904788,1.171206,3947.851834,2.194520,439251.0
2020-02-10 04:00:00,10120.40,10120.40,10060.00,10090.47,1750.261272,2020-02-10 04:00:00,2020-02-10 03:00:00,10135.542993,1.170725,3948.439069,2.192636,439252.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2024-02-10 19:00:00,47196.49,47337.92,47182.31,47329.37,604.318580,2024-02-10 19:00:00,2024-02-10 18:00:00,25676.252503,1.162726,25200.157814,0.876296,474331.0
2024-02-10 20:00:00,47329.38,47660.00,47304.72,47531.09,1413.680920,2024-02-10 20:00:00,2024-02-10 19:00:00,25676.471768,1.162459,25200.217899,0.876503,474332.0
2024-02-10 21:00:00,47531.09,48170.00,47454.37,47878.00,2793.799670,2024-02-10 21:00:00,2024-02-10 20:00:00,25676.911957,1.162164,25200.278062,0.876739,474333.0


In [60]:
acf_value= acf(df.close.head(p))
pacf_value = pacf(df.close.head(p))

y = acf_value[1:].tolist()       # [0:z]
y_= pacf_value[1:].tolist()
y.reverse()
y_.reverse()
z=len(y)

x=[]
for i in range(0,z):
    x.append(i)
    
close1 = df.close[(p-z):p]   #рост
# close1 = df.close.tail(z)

data= pd.DataFrame({'x':x, 'y':y, 'y_':y_,'close':close1})    ################!!!!!!!!!!!!!!тут меняем close для калибровки
data['modeling']=data.y*data.close
data['modeling_']=data.modeling*data.y_
zn = 45
# plt.plot(data.x.tail(zn),data.modeling.tail(zn))  #визуализация для лохов

In [61]:
#Последние два значения перед прогнозируемым
x_train = data.x.tail(zn).head(zn-1)
y_train = data.modeling.tail(zn).head(zn-1)

y_test = df.close[p:(p+1)].item()
x_test=pd.DataFrame({45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80})
# x_test

In [62]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression().fit(x_train.values.reshape(-1,1), y_train.values.reshape(-1,1))
pred= lr.predict(x_test.values.reshape(-1,1))

delta = abs(y_train[-1]-pred)
deltasqr=abs(y_train[-1]-pred)
deltadf = pd.DataFrame({'delta':delta.flatten(), 'pred':pred.flatten()})
deltadf
# print(f'delta = {delta}')
# print(f'pred = {pred}\n')
# print(f'y_test = {y_test} ')
# print(f'precision={100/(y_test/pred[0])}')
# print(metrix.mae(y_test, pred))
# print(metrix.rmse(y_test, pred))

,delta,pred
0,232.727956,47691.715484
1,200.338356,47724.105085
2,167.948755,47756.494686
3,135.559154,47788.884286
4,103.169554,47821.273887
5,70.779953,47853.663488
6,38.390352,47886.053088
7,6.000752,47918.442689
8,26.388849,47950.832290
9,58.778449,47983.221890


2.856805503644864
2.856805503644864
precision=[99.99406597]
y_test = 48142.78, pred = [[48139.9231945]]     НАСТРОЕННОЕ ПЕРВИЧНОЕ ЗНАЧЕНИЕ

79.49877820649272
79.49877820649272
precision=[99.83359485]
y_test = 47774.23, pred = [[47694.73122179]]     ТЕСТОВОЕ ЗНАЧЕНИЕ zn = 45    при дельта 20 можно использовать прогноз

In [ ]:
73.76714365826774
73.76714365826774
precision=[99.84559219]
y_test = 47774.23, pred = [[47700.46285634]]    zn=45

протестировать на другой выборке
ЧТО ЕСЛИ ИЗЗА ТОГО ЧТО Я ДАЛ ЗНАЧЕНИЕ ГОТОВОЕ В ACF   


добавить для точности PACF